# SparkFund

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [18]:
rounds = pd.read_csv("Dataset/rounds2.csv",encoding='iso-8859-1 ')
companies = pd.read_csv("Dataset/companies.csv",encoding='iso-8859-1')
companies.head()
rounds.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [20]:
companies.describe()

,company_permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
count,18334,18334,16941,17488,18333,16531,16085,16240,16240,14112
unique,18334,18272,16916,8208,4,107,251,779,2586,2171
top,/Organization/Artoo,Blink,http://www.askforoffer.com,Biotechnology,operating,USA,CA,SF Bay Area,San Francisco,01-01-2012
freq,1,4,3,1235,14592,10671,3606,2428,911,731


###  How many unique companies are present in rounds2?